In [1]:
# from keras.layers import Input, Dense
# from keras.models import Model
# from keras import backend
import tensorflow as tf
import keras
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Layer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.eager.context import context, EAGER_MODE, GRAPH_MODE


# from keras.models import Sequential
# from keras.layers import Dense, Activation
# from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
loaded_x_train = pd.read_csv("./Dataset/x_train.csv")
loaded_y_train = pd.read_csv("./Dataset/y_train.csv")
loaded_x_test = pd.read_csv("./Dataset/x_test.csv")
loaded_y_test = pd.read_csv("./Dataset/y_test.csv")

In [3]:
print(loaded_x_train.shape)
print(loaded_x_test.shape)
print(loaded_y_train.shape)
print(loaded_y_test.shape)

(370515, 120)
(123506, 120)
(370515, 23)
(123506, 23)


In [4]:
loaded_x_train_numpy = np.array(loaded_x_train)
loaded_x_test_numpy = np.array(loaded_x_test)
loaded_y_train_numpy = np.array(loaded_y_train)
loaded_y_test_numpy = np.array(loaded_y_test)

In [39]:
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
import qiskit
from qiskit.aqua.operators import Z
from qiskit.aqua.operators import StateFn
quantumRegister = QuantumRegister(4)
quantumCircuit = QuantumCircuit(quantumRegister)
backend = qiskit.Aer.get_backend("qasm_simulator")
operatorZ = Z ^ Z ^ Z ^ Z ^ Z ^ Z ^ Z ^ Z

In [6]:
from qiskit import IBMQ
from qiskit.providers.ibmq import least_busy

In [7]:
provider = IBMQ.load_account()
device = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 8 and 
                                   not x.configuration().simulator and x.status().operational==True))
print(device)

ibmq_16_melbourne


In [41]:
import time
def quantum_layer(initial_parameters, rotation_parameters):
    # expecting parameters to be a numpy array

    quantumRegister = QuantumRegister(8)
    quantumCircuit = QuantumCircuit(quantumRegister)
    backend = qiskit.Aer.get_backend("qasm_simulator")
    index = 0
    quantumCircuit.h(range(8))

    for i in range(len(initial_parameters)):
        quantumCircuit.ry(initial_parameters[i] * np.pi, i)

    while (index <= 7):

        parameter_1 = rotation_parameters[index]
        qubit_one = index
        index = index + 1
        parameter_2 = rotation_parameters[index]
        qubit_two = index
        index = index + 1

        quantumCircuit.cry(2 * parameter_1 * np.pi, qubit_one, qubit_two)
        quantumCircuit.cry(2 * parameter_2 * np.pi, qubit_two, qubit_one)
    
    
    
    psi = StateFn(quantumCircuit)
    expectationZ = (~psi @ operatorZ @ psi).eval()
    # expectationZ = psi.adjoint().compose(operatorZ).compose(psi).eval().real
    print(expectationZ)
    
#     expectations = [200* np.abs(np.real(expectationZ)), 400*np.abs(np.real(expectationZ)), 600*np.abs(np.real(expectationZ)),
#                     800 * np.abs(np.real(expectationZ)), 1000 * np.abs(np.real(expectationZ)), 300 * np.abs(np.real(expectationZ)),
#                     500 * np.abs(np.real(expectationZ)), 700 * np.abs(np.real(expectationZ)) ]

    quantumCircuit.measure_all()
    result = qiskit.execute(quantumCircuit, backend, shots=1000).result()
    counts = result.get_counts(quantumCircuit)

    del(quantumCircuit)
    del(quantumRegister)
    # print("Exp input {input}, w {w}, expectation {expectation}".format(input=np.mean(initial_parameters), w=np.mean(
    #     rotation_parameters), expectation=np.mean(expectation*10000)
    #  ))


    return counts


def quantum_operation(initial_parameters, rotation_parameters):
    final_output = []
    for i in range(initial_parameters.shape[0]):
        pred = quantum_layer(initial_parameters[i], rotation_parameters[i])
        final_output.append(list(pred))
    return final_output

In [10]:
print(type(operatorZ))

<class 'qiskit.aqua.operators.primitive_ops.pauli_op.PauliOp'>


In [13]:
from qiskit.quantum_info.analysis.average import average_data

In [42]:
counts = quantum_layer([0.2, 0.2, 0.1, 0.6, 0.7, 0.5, 0.1, 0.8], [0.2, 0.2, 0.1, 0.6, 0.7, 0.5, 0.1, 0.8])

ValueError: Composition is not defined over Operators of different dimensions, 16 and 8, respectively.

In [32]:
average_data(counts, [Z, Z, Z, Z, Z, Z, Z, Z])

0

In [40]:
operatorZ

PauliOp(Pauli(z=[True, True, True, True, True, True, True, True], x=[False, False, False, False, False, False, False, False]), coeff=1.0)